In [ ]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler


from tabtransformertf.models.fttransformer import FTTransformerEncoder, FTTransformer
from tabtransformertf.utils.preprocessing import df_to_dataset, build_categorical_prep

Data Preparation

Model settings

In [ ]:
from tabtransformertf.models.fttransformer import Time2Vec
t2v_encoder = Time2Vec(
    time_features=TIME_FEATURES,
    kernel_size=16,
    t2v_emb_dim=16
)

In [ ]:
ft_linear_encoder = FTTransformerEncoder(
    numerical_features = NUMERIC_FEATURES,
    categorical_features = [],
    numerical_data = X_train[NUMERIC_FEATURES].values,
    categorical_data = None, 
    y = None,
    numerical_embedding_type='linear',
    embedding_dim=128,
    depth=4,
    heads=8,
    attn_dropout=0.2,
    ff_dropout=0.2,
    explainable=True
)

# Pass th encoder to the model
ft_linear_model = FTTransformer(
    encoder=ft_linear_encoder,
    t2v_encoder=t2v_encoder,
    out_dim=1,
    out_activation="relu",
)

LEARNING_RATE = 5e-4
WEIGHT_DECAY = 0.0001
NUM_EPOCHS = 1000

optimizer = tfa.optimizers.AdamW(
        learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )

ft_linear_model.compile(
    optimizer = optimizer,
    loss = {"output": MyHuberLoss(threshold=0.05), "importances": None},
    metrics= {"output": [tf.keras.metrics.RootMeanSquaredError(name='rmse')], "importances": None},
)

early = EarlyStopping(monitor="val_output_loss", mode="min", patience=20, restore_best_weights=True)
callback_list = [early]
ft_linear_history = ft_linear_model.fit(
    train_dataset, 
    epochs=NUM_EPOCHS, 
    validation_data=val_dataset,
    callbacks=callback_list
)